In [56]:
"""
Shows SpamAssassin Python Wrapper at work.

https://pypi.org/project/spamassassin-client/


- SETUP: -
1) Have spamasassin installed
    > sudo apt get spamassassin
2) > pip install spamassassin_client
3) Start spamassassin server
    > sudo spamd
4) Use spamassassin_client

- NOTE: -
REMEMBER TO SHUT DOWN SPAMD SERVER WHEN DONE

"""
import os
from spamassassin_client import SpamAssassin
import pandas as pd
import numpy as np
import util
import re
import os

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
# only get plain text emails from corpus
corpus_spam_path = "data/spam_assassin_corpus/spam_2"
corpus_ham_path = "data/spam_assassin_corpus/easy_ham"

In [58]:
def is_plaintext(email):
    header = email.split("\n\n", 1)[0].strip()
    if "content-type: text/plain" in header.lower():
        return True
    return False

In [59]:
# iterate over files in spam folder
plaintext_spam = []
for filename in os.listdir(corpus_spam_path):
    with open(os.path.join(corpus_spam_path, filename), "r") as f:
        try:
            email = f.read()
            if is_plaintext(email):
                plaintext_spam.append((filename, email))
        except:
            continue
len(plaintext_spam)

347

In [60]:
# iterate over files in ham folder
plaintext_ham = []
for filename in os.listdir(corpus_ham_path):
    with open(os.path.join(corpus_ham_path, filename), "r") as f:
        try:
            email = f.read()
            if is_plaintext(email):
                plaintext_ham.append((filename, email))
        except:
            continue
len(plaintext_ham)

1891

In [61]:
CLEAN_HEADER = """Subject: {subject}
Message-ID: <GTUBE1.1010101@example.net>
Date: Wed, 23 Jul 2003 23:30:00 +0200
From: Sender <sender@example.net>
To: Recipient <recipient@example.net>
MIME-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit"""

def replace_header(email):
    spl = email.split("\n\n", 1)
    header = spl[0].strip()
    body = spl[1].strip()

    subj = re.match(r'(.|\n)*subject:(.*)', header, flags = re.IGNORECASE).group(2).strip()
    return CLEAN_HEADER.format(subject=subj) + "\n\n" + body

In [62]:
cleaned_spam = [replace_header(email) for fn, email in plaintext_spam]
cleaned_ham = [replace_header(email) for fn, email in plaintext_ham]